In [1]:
DF = spark.read.option("sep", ",").option("header", "true").csv("CrabAgePrediction.csv")
DF.printSchema()
DF.show()

root
 |-- Sex: string (nullable = true)
 |-- Length: string (nullable = true)
 |-- Diameter: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Shucked Weight: string (nullable = true)
 |-- Viscera Weight: string (nullable = true)
 |-- Shell Weight: string (nullable = true)
 |-- Age: string (nullable = true)

+---+------+--------+------+-----------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|     Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+-----------+--------------+--------------+------------+---+
|  F|1.4375|   1.175|0.4125| 24.6357155|    12.3320325|     5.5848515|    6.747181|  9|
|  M|0.8875|    0.65|0.2125| 5.40057975|     2.2963095|    1.37495075|   1.5592225|  6|
|  I|1.0375|   0.775|  0.25| 7.95203475|      3.231843|    1.60174675|  2.76407625|  6|
|  F| 1.175|  0.8875|  0.25|13.48018725|    4.74854125|    2.28213475|   5.2446575| 10|
|  I|0.8875|  0.66

In [3]:
weights=[0.7, 0.3]
train, test = DF.randomSplit(weights)
train.show(3)
test.show(3)

+---+------+--------+------+----------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|    Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+----------+--------------+--------------+------------+---+
|  F|0.7625|  0.5625| 0.175|4.20990075|    1.65844575|    0.94970825|   1.2757275|  7|
|  F|0.8125|    0.65| 0.225|5.42892925|     2.4097075|      1.020582|    1.757669|  7|
|  F| 0.825|    0.65|   0.2|    5.6699|    1.77184375|      1.417475|    1.984465|  9|
+---+------+--------+------+----------+--------------+--------------+------------+---+
only showing top 3 rows

+---+------+--------+------+---------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|   Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+---------+--------------+--------------+------------+---+
|  F|0.6875|  0.4875| 0.175|  2.26796|     0.8788345|    0.60951425|   0.7087375|  5|
|  F| 0.725|   0.525|0

In [10]:
print(train.count())
print(test.count())

2770
1123


In [15]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

In [26]:
import numpy as np

# Преобразуйте PySpark DataFrame в pandas DataFrame
DF_pd = DF.toPandas()

# Преобразуйте pandas DataFrame в массив numpy
df = DF_pd.values


In [32]:
X = np.delete(df, 7, axis =1)  # Признаки
y = df['Age']  # Целевая переменная
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7)
knn = KNeighborsRegressor(n_neighbors=5)  # Создание модели с количеством соседей равным 5 (можно выбрать другое значение)
knn.fit(X_train, y_train)  # Обучение модели на обучающем наборе данных
score = knn.score(X_test, y_test)
print("Оценка результата:", score)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [31]:
print(df)

[['F' '1.4375' '1.175' ... '5.5848515' '6.747181' '9']
 ['M' '0.8875' '0.65' ... '1.37495075' '1.5592225' '6']
 ['I' '1.0375' '0.775' ... '1.60174675' '2.76407625' '6']
 ...
 ['I' '0.625' '0.4625' ... '0.52446575' '0.63786375' '5']
 ['I' '1.0625' '0.775' ... '2.33883375' '2.9766975' '6']
 ['I' '0.7875' '0.6125' ... '1.34660125' '1.417475' '8']]
